# The Email Assistant Agent

> Note: This notebook demonstrates the final agent. See `langgraph_101.ipynb` for a simpler introduction to LangGraph concepts.

This notebook shows how to run our email assistant, which combines multiple features we've built: a triage router, a response agent with tools, human-in-the-loop (HITL) capabilities, and a persistent reminder system.

![overview-img](img/overview.png)

### Live Gmail Setup (optional)

If you plan to test with real Gmail, unset eval/demo flags so the agent behaves live, and ensure credentials are configured (see `src/email_assistant/tools/gmail/README.md`).

- Unset `EMAIL_ASSISTANT_EVAL_MODE` to disable synthetic tool plans.
- Optionally unset `HITL_AUTO_ACCEPT` to review drafts in Agent Inbox.
- Optionally unset `EMAIL_ASSISTANT_SKIP_MARK_AS_READ` so threads are marked read.
- Leave `EMAIL_ASSISTANT_RECIPIENT_IN_EMAIL_ADDRESS` unset for correct live sending semantics.

### Live-First Workflow Checklist

- **Start LangGraph Studio**: In a terminal, run `langgraph up` and pick `email_assistant_hitl_memory_gmail`. This launches Agent Inbox for HITL approvals and exposes the Reminder tab so you can watch follow-ups being created.
- **Provide live credentials**: Export `GOOGLE_API_KEY` (Gemini) and set `GEMINI_MODEL=gemini-2.5-pro`. Leave `EMAIL_ASSISTANT_EVAL_MODE` unset so the notebook exercises the real agent path.
- **Run the live smoke tests first**:
  ```bash
  ! pytest tests/test_live_smoke.py --agent-module email_assistant_hitl_memory_gmail
  ! pytest tests/test_live_hitl_spam.py --agent-module email_assistant_hitl_memory_gmail
  ```
  These mirror what Studio triggers—only fall back to the offline toggles once live verification passes (`EMAIL_ASSISTANT_EVAL_MODE=1`, `HITL_AUTO_ACCEPT=1`, `EMAIL_ASSISTANT_SKIP_MARK_AS_READ=1`, optional `EMAIL_ASSISTANT_RECIPIENT_IN_EMAIL_ADDRESS=1`).
- **Reminder worker**: The flow below expects the background worker running. In another shell start `python scripts/reminder_worker.py --loop` (or `--once` during demos) and configure the reminder env vars (`REMINDER_DEFAULT_HOURS`, `REMINDER_POLL_INTERVAL_MIN`, `REMINDER_DB_PATH`, `REMINDER_NOTIFY_EMAIL`, `REMINDER_LABEL_*`).

In [ ]:
# Uncomment for live Gmail behavior inside Studio/Agent Inbox runs.
# os.environ.pop("EMAIL_ASSISTANT_EVAL_MODE", None)
# os.environ["HITL_AUTO_ACCEPT"] = "0"  # review drafts in Agent Inbox
# os.environ.pop("EMAIL_ASSISTANT_SKIP_MARK_AS_READ", None)
# os.environ.pop("EMAIL_ASSISTANT_RECIPIENT_IN_EMAIL_ADDRESS", None)

# Tip: ensure GOOGLE_API_KEY and Gmail credentials are set per tools/gmail/README.

> Update: See `notebooks/UPDATES.md` for environment flags, Gmail HITL display details, and the structured outputs now returned by the Gmail agent.

### 1. Load Environment Variables

In [ ]:
import os
from dotenv import load_dotenv

# Load from .env file in the parent directory
load_dotenv("../.env")

# Default to auto-accept in notebooks/tests but allow override
os.environ.setdefault("HITL_AUTO_ACCEPT", "1")


### 2. Import and Run the Agent

All of our agent logic has been modularized into the `src/email_assistant` directory. We can now import the final, compiled agent directly.

In [ ]:
import sys
import uuid
# Add src to path to allow for imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from email_assistant.email_assistant_hitl_memory_gmail import email_assistant

In [ ]:
# Here is an example of an email requiring a response
email_input = {
  "id": "thread_abc_123",
  "from": "Alice <alice@example.com>",
  "to": "me@example.com",
  "subject": "Quick question about API documentation",
  "body": "Hi,\nI was reviewing the API documentation and had a question about the auth endpoint.\nThanks!\nAlice"
}

# The config is necessary to provide a unique thread_id for the checkpointer
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

# Run the agent
response = email_assistant.invoke({"email_input": email_input}, config)

# Show the agent's final assistant message (if any)
print(response.get('messages', [])[-1].content)

# Also show the structured outputs that evaluators may expect
print("\nassistant_reply:\n", response.get('assistant_reply', ''))
tool_trace = response.get('tool_trace', '')
if tool_trace:
    # Print a trimmed view for readability
    print("\ntool_trace (first 800 chars):\n", tool_trace[:800])
else:
    print("\ntool_trace: (empty)")

### 3. Advanced Feature: Reminders & Follow-ups

We have added a persistent reminder system to the agent. The reminder worker in `scripts/reminder_worker.py` polls for pending follow-ups and applies the Gmail labels configured by:

- `REMINDER_DEFAULT_HOURS`, `REMINDER_POLL_INTERVAL_MIN`
- `REMINDER_DB_PATH`, `REMINDER_NOTIFY_EMAIL`
- `REMINDER_LABEL_PENDING`, `REMINDER_LABEL_DUE`, `REMINDER_LABEL_DONE`

Start the worker alongside the notebook to see reminders transition states:

```bash
! python scripts/reminder_worker.py --loop
```

Use `--once` for single runs or when demonstrating in eval mode. The cells below replay the same reminder emails used in the automated tests so Notebook users can verify the end-to-end experience.

#### Step A: Clean the Database

First, let's ensure the reminder database is empty so we start from a clean state.

In [ ]:
db_path = os.getenv("REMINDER_DB_PATH", "../.local/reminders.db")
if os.path.exists(db_path):
    os.remove(db_path)
    print(f"Removed existing database at {db_path}")

#### Step B: Create a Reminder

We'll process an important email. The agent should classify it as `respond` or `notify` and create a reminder in the database.

In [ ]:
import json

with open('../tests/evaluation_data/reminders/01_create_reminder.txt', 'r') as f:
    create_email_data = json.load(f)

config = {"configurable": {"thread_id": str(uuid.uuid4()), "recursion_limit": 60}}
email_assistant.invoke({"email_input": create_email_data}, config)

#### Step C: Verify Reminder Creation

We can now use the `reminder_worker.py` script's `--list` command to see the reminder that was just created. We use the `!` prefix to run shell commands from the notebook.

In [ ]:
!python ../scripts/reminder_worker.py --list

#### Step D: Cancel the Reminder

Next, we process a reply from the user in the same thread. The agent should detect this reply and automatically cancel the pending reminder.

In [ ]:
with open('../tests/evaluation_data/reminders/02_cancel_reminder.txt', 'r') as f:
    cancel_email_data = json.load(f)

config = {"configurable": {"thread_id": str(uuid.uuid4()), "recursion_limit": 60}}
email_assistant.invoke({"email_input": cancel_email_data}, config)

#### Step E: Verify Reminder Cancellation

Finally, we list the reminders again. The list should now be empty.

In [ ]:
!python ../scripts/reminder_worker.py --list